# Census Notebook
## Doing' it Dask-y Style!

Held every 10 years, the US census gives a detailed snapshot in time about the makeup of the country.  The last census in 2010 surveyed nearly 309 million people.  IPUMS.org provides researchers an open source data set with 1% to 10% of the census data set.  In this notebook, we want to see how education affects total income earned in the US based on data from each census from the 1970 to 2010 and see if we can predict some results if the census was held today, according to the national average.  We will go through the ETL, training the model, and then testing the prediction.  We'll make every effort to get as balanced of a dataset as we can.  We'll also pull some extra variables to allow for further self-exploration of gender based education and income breakdowns.  On a single Titan RTX, you can run the whole notebook workflow on the 4GB dataset of 14 million rows by 44 columns in less than 3 minutes.  

**Let's begin!**

## Imports

In [6]:
import pandas as pd
import numpy as np
import cuml
import cudf
import dask_cudf
import dask_cuml
import sys
import os
import gzip
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

## Get your data!

In [7]:
# for those of us using dask
!pip install requests
"""dtype={'YEAR': 'float64',
       'DATANUM': 'float64',
       'CBSERIAL': 'float64',
       'HHWT': 'float64',
       'CPI99': 'float64',
       'GQ': 'float64',
       'QGQ': 'float64',
       'PERNUM': 'float64',
       'PERWT': 'float64',
       'SEX': 'float64',
       'AGE': 'float64',
       'EDUC': 'float64',
       'EDUCD': 'float64',
       'INCTOT': 'float64',
       'SEX_HEAD': 'float64',
       'SEX_MOM': 'float64',
       'SEX_POP': 'float64',
       'SEX_SP': 'float64',
       'SEX_MOM2': 'float64',
       'SEX_POP2': 'float64',
       'AGE_HEAD': 'float64',
       'AGE_MOM': 'float64',
       'AGE_POP': 'float64',
       'AGE_SP': 'float64',
       'AGE_MOM2': 'float64',
       'AGE_POP2': 'float64',
       'EDUC_HEAD': 'float64',
       'EDUC_MOM': 'float64',
       'EDUC_POP': 'float64',
       'EDUC_SP': 'float64',
       'EDUC_MOM2': 'float64',
       'EDUC_POP2': 'float64',
       'EDUCD_HEAD': 'float64',
       'EDUCD_MOM': 'float64',
       'EDUCD_POP': 'float64',
       'EDUCD_SP': 'float64',
       'EDUCD_MOM2': 'float64',
       'EDUCD_POP2': 'float64',
       'INCTOT_HEAD': 'float64',
       'INCTOT_MOM': 'float64',
       'INCTOT_POP': 'float64',
       'INCTOT_SP': 'float64',
       'INCTOT_MOM2': 'float64',
       'INCTOT_POP2': 'float64']"""

df = dask_cudf.read_csv('https://rapidsai-data.s3.us-east-2.amazonaws.com/datasets/ipums_education2income_1970-2010.csv.gz', assume_missing=True, compression='gzip')
print('data',df.shape)

data (Delayed('int-dc8804f3-cde4-428b-a37d-bb37cdc7808a'), 45)


In [8]:
print(df.head(5).to_pandas())

     YEAR  DATANUM  SERIAL  CBSERIAL   HHWT  CPI99   GQ  QGQ  PERNUM  PERWT  \
0  1970.0      2.0     1.0       NaN  100.0   4.54  1.0  0.0     1.0  100.0   
1  1970.0      2.0     1.0       NaN  100.0   4.54  1.0  0.0     2.0  100.0   
2  1970.0      2.0     2.0       NaN  100.0   4.54  1.0  0.0     1.0  100.0   
3  1970.0      2.0     2.0       NaN  100.0   4.54  1.0  0.0     2.0  100.0   
4  1970.0      2.0     4.0       NaN  100.0   4.54  1.0  0.0     1.0  100.0   

      ...       EDUCD_POP  EDUCD_SP  EDUCD_MOM2  EDUCD_POP2  INCTOT_HEAD  \
0     ...             NaN      30.0         NaN         NaN      12450.0   
1     ...             NaN      60.0         NaN         NaN      12450.0   
2     ...             NaN      60.0         NaN         NaN       9050.0   
3     ...             NaN      70.0         NaN         NaN       9050.0   
4     ...             NaN      23.0         NaN         NaN       7450.0   

   INCTOT_MOM  INCTOT_POP  INCTOT_SP  INCTOT_MOM2  INCTOT_POP2  
0  

In [9]:
df.dtypes

YEAR           float64
DATANUM        float64
SERIAL         float64
CBSERIAL       float64
HHWT           float64
CPI99          float64
GQ             float64
QGQ            float64
PERNUM         float64
PERWT          float64
SEX            float64
AGE            float64
EDUC           float64
EDUCD          float64
INCTOT         float64
SEX_HEAD       float64
SEX_MOM        float64
SEX_POP        float64
SEX_SP         float64
SEX_MOM2       float64
SEX_POP2       float64
AGE_HEAD       float64
AGE_MOM        float64
AGE_POP        float64
AGE_SP         float64
AGE_MOM2       float64
AGE_POP2       float64
EDUC_HEAD      float64
EDUC_MOM       float64
EDUC_POP       float64
EDUC_SP        float64
EDUC_MOM2      float64
EDUC_POP2      float64
EDUCD_HEAD     float64
EDUCD_MOM      float64
EDUCD_POP      float64
EDUCD_SP       float64
EDUCD_MOM2     float64
EDUCD_POP2     float64
INCTOT_HEAD    float64
INCTOT_MOM     float64
INCTOT_POP     float64
INCTOT_SP      float64
INCTOT_MOM2

In [10]:
original_counts = df.YEAR.value_counts()
print(original_counts) ### Remember these numbers!

<dask_cudf.Series | 6 tasks | 1 npartitions>


## ETL

### Cleaning Income data
First, let's focus on cleaning out the bad values for Total Income `INCTOT`. First, let's see if there are an `N/A` values, as when we did `head()`, we saw some in other columns, like CBSERIAL

In [11]:
df['INCTOT_NA'] = df['INCTOT'].isna()

In [12]:
print(df.INCTOT_NA.value_counts())

<dask_cudf.Series | 9 tasks | 1 npartitions>


Okay, great, there are no `N/A`s...or are there?  Let's drop `INCTOT_NA` and see what our value counts look like

In [13]:
df=df.drop('INCTOT_NA')
print(df.INCTOT.value_counts().to_pandas())  ### Wow, look how many people in America make $10,000,000!  Wait a minutes... 

NotImplementedError: Drop currently only works for axis=1

Not that many people make $10M a year. Checking https://usa.ipums.org/usa-action/variables/INCTOT#codes_section, `9999999`is INCTOT's code for `N/A`.  That was why when we ran `isna`, RAPIDS won't find any.  Let's first create a new dataframe that is only NA values, then let's pull those encoded `N/A`s out of our working dataframe!

In [14]:
print('data',df.shape)
tdf = df.query('INCTOT == 9999999')
df = df.query('INCTOT != 9999999')

data (Delayed('int-751d1b42-b1e5-4452-b42b-9e4e577cebc8'), 46)


In [15]:
print('working data',df.shape)
print('junk count data',tdf.shape)

working data (Delayed('int-71c4905f-4db3-4431-8a68-a86e1e11c5c8'), 46)
junk count data (Delayed('int-57901cce-2bc1-4c06-80d3-bf7529deac03'), 46)


We're down by nearly 1/4 of our original dataset size.  For the curious, now we should be able to get accurate Total Income data, by year, not taking into account inflation

In [16]:
print(df.groupby('YEAR')['INCTOT'].mean()) # without that cleanup, the average would have bene in the millions....

<dask_cudf.DataFrame | 13 tasks | 1 npartitions>


#### Normalize Income for inflation
Now that we have reduced our dataframe to a baseline clean data to answer our question, we should normalize the amounts for inflation.  `CPI99`is the value that IPUMS uses to contian the inflation factor.  All we have to do is multipy by year.  Let's see how that changes the Total Income values from just above!

In [19]:
print(df.groupby('YEAR')['CPI99'].mean()) ## it just returns the CPI99
df['INCTOT'] = df['INCTOT'] * df['CPI99']
print(df.groupby('YEAR')['INCTOT'].mean()) ## let's see what we got!
df.compute()

<dask_cudf.DataFrame | 21 tasks | 1 npartitions>
<dask_cudf.DataFrame | 25 tasks | 1 npartitions>


<cudf.DataFrame ncols=46 nrows=16833597 >

### Cleaning Education Data
Okay, great!  Now we have income cleaned up, it should also have cleaned much of our next sets of values of interes, namely Education and Education Detailed.  However, there are still some `N/A`s in key variables to worry about, which can cause problmes later.  Let's create a list of them...

In [20]:
suspect = ['CBSERIAL','EDUC', 'EDUCD', 'EDUC_HEAD', 'EDUC_POP', 'EDUC_MOM','EDUCD_MOM2','EDUCD_POP2', 'INCTOT_MOM','INCTOT_POP','INCTOT_MOM2','INCTOT_POP2', 'INCTOT_HEAD']

In [21]:
for i in range(0, len(suspect)):
    df[suspect[i]] = df[suspect[i]].fillna(-1)
    print(suspect[i], df[suspect[i]].value_counts())

CBSERIAL <dask_cudf.Series | 25 tasks | 1 npartitions>
EDUC <dask_cudf.Series | 28 tasks | 1 npartitions>
EDUCD <dask_cudf.Series | 31 tasks | 1 npartitions>
EDUC_HEAD <dask_cudf.Series | 34 tasks | 1 npartitions>
EDUC_POP <dask_cudf.Series | 37 tasks | 1 npartitions>
EDUC_MOM <dask_cudf.Series | 40 tasks | 1 npartitions>
EDUCD_MOM2 <dask_cudf.Series | 43 tasks | 1 npartitions>
EDUCD_POP2 <dask_cudf.Series | 46 tasks | 1 npartitions>
INCTOT_MOM <dask_cudf.Series | 49 tasks | 1 npartitions>
INCTOT_POP <dask_cudf.Series | 52 tasks | 1 npartitions>
INCTOT_MOM2 <dask_cudf.Series | 55 tasks | 1 npartitions>
INCTOT_POP2 <dask_cudf.Series | 58 tasks | 1 npartitions>
INCTOT_HEAD <dask_cudf.Series | 61 tasks | 1 npartitions>


Let's get drop any rows of any `-1`s in Education and Education Detailed.

In [22]:
totincome = ['EDUC','EDUCD']
for i in range(0, len(totincome)):
    query = totincome[i] + ' != -1'
    df = df.query(query)
    print(totincome[i])

EDUC
EDUCD


In [ ]:
print(df.shape)
df.head().to_pandas().head()

(Delayed('int-6622dc1c-5aa7-4b4a-ac91-4dcc7ccee0f3'), 46)


Well, the good news is that we lost no further rows, start to normalize the data so when we do our OLS, one year doesn't unfairly dominate the data

## Normalize the Data
The in the last step, need to keep our data at about the same ratio as we when started (1% of the population), with the exception of 1980, which was a 5% and needs to be reduced.  This is why we kept the temp dataframe `tdf` - to get the counts per year.   we will find out just how many have to realize

In [ ]:
print('Working data: \n', df.YEAR.value_counts())
print('junk count data: \n', tdf.YEAR.value_counts())

And now, so that we can do MSE, let's make all the dtypes the same. 

In [ ]:
df.dtypes

In [ ]:

keep_cols = ['YEAR', 'DATANUM', 'SERIAL', 'CBSERIAL', 'HHWT', 'GQ', 'PERNUM', 'SEX', 'AGE', 'INCTOT', 'EDUC', 'EDUCD', 'EDUC_HEAD', 'EDUC_POP', 'EDUC_MOM','EDUCD_MOM2','EDUCD_POP2', 'INCTOT_MOM','INCTOT_POP','INCTOT_MOM2','INCTOT_POP2', 'INCTOT_HEAD', 'SEX_HEAD']
df = df.loc[:, keep_cols]
#df = df.drop(col for col in df.columns if col not in keep_cols)
for i in range(0, len(keep_cols)):
    df[keep_cols[i]] = df[keep_cols[i]].fillna(-1)
    print(keep_cols[i], df[keep_cols[i]].value_counts())

In [ ]:
## I WANTED TO REDUCE THE 1980 SAMPLE HERE, BUT .SAMPLE() IS NEEDED AND NOT WORKING, UNLESS THERE IS A WORK AROUND...

With the important data now clean and normalized, let's start doing the regression

## Ridge Regression
We have 44 variables.  The other variables may provide important predictive information.  The Ridge Regression technique with cross validation to identify the best hyperparamters may be the best way to get the most accurate model.  We'll have to 

* define our performance metrics
* split our data into train and test sets
* train and test our model

Let's begin and see what we get!

In [ ]:
# As our performance metrics we'll use a basic mean squared error and coefficient of determination implementation
def mse(y_test, y_pred):
    return ((y_test - y_pred) ** 2).mean()

def cod(y_test, y_pred):
    y_bar = y_test.mean()
    total = ((y_test - y_bar) ** 2).sum()
    residuals = ((y_test - y_pred) ** 2).sum()
    return 1 - (residuals / total)

In [ ]:
from cuml.preprocessing.model_selection import train_test_split
trainsize = .9
yCol = "EDUC"
from cuml.preprocessing.model_selection import train_test_split
from cuml.linear_model.ridge import Ridge

def train_and_score(data, clf, train_frac=0.8, n_runs=20):
    mse_scores, cod_scores = [], []
    for _ in range(n_runs):
        X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(df, yCol, train_size=.9)
        y_pred = clf.fit(X_train, y_train).predict(X_test)
        mse_scores.append(mse(y_test, y_pred))
        cod_scores.append(cod(y_test, y_pred))
    return mse_scores, cod_scores

 ## Results
 **Moment of truth!  Let's see how our regression training does!**

In [ ]:
import numpy as np
n_runs = 20
clf = Ridge()
mse_scores, cod_scores = train_and_score(df, clf, n_runs=n_runs)
print(f"median MSE ({n_runs} runs): {np.median(mse_scores)}")
print(f"median COD ({n_runs} runs): {np.median(cod_scores)}")

**Fun fact:** if you made INCTOT the y axis, your prediction results would not be so pretty!  It just shows that your education level can be an indicator for your income, but your income is NOT a great predictor for your education level.  You have better odds flipping a coin!

* median MSE (50 runs): 518189521.07548225
* median COD (50 runs): 0.425769113846303

## Next Steps/Self Study
* You can pickle the model and use it in another workflow
* You can redo the workflow with based on head of household using `EDUC`, `SEX`, and `INCTOT` for X in `X`_HEAD
* You can see the growing role of education with women in their changing role in the workforce and income with "EDUC_MOM" and "EDUC_POP